# C1: Selection vs Representation Failure Decomposition

**Paper:** Goldstone Modes and the Coexistence Saddle (v7 revision)

Both external reviewers independently identified this as the missing piece: run the selection vs representation failure decomposition rather than deferring it to future work.

**Diagnostic:** At trial end, if both networks retain above-threshold activity (max_r > 0.3), a swap error is classified as **selection failure** (both representations survived but the wrong one was decoded). If one network collapsed (max_r < 0.1), it is classified as **representation failure** (one item was lost).

**Expected result:** Valley regime (J_× ≈ 1.2–1.6) should show predominantly selection-type failures; near-critical regime (J_× ≈ 0.3–0.5) should show representation-type.

In [ ]:
# Cell 1: Model infrastructure (self-contained, no imports from local files)
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import i0
import json, time

# === MODEL PARAMETERS (match paper §2.1–2.2) ===
N = 48
J_0, J_1 = 1.0, 6.0
BETA, H0, R_MAX = 5.0, 0.5, 1.0
TAU, DT = 10.0, 1.0
KAPPA = 2.0
NOISE_SIGMA = 0.1       # ← CHECK: paper says 0.1; sweep code used 0.3
N_STEPS = 500
STIM_STEPS = 100
SEPARATION = np.pi / 2  # 90° between items

PREFERRED = np.linspace(-np.pi, np.pi, N, endpoint=False)

def sigmoid(h):
    return R_MAX / (1.0 + np.exp(-BETA * (h - H0)))

def tuning_curve(theta, preferred, kappa):
    return np.exp(kappa * np.cos(theta - preferred)) / (2 * np.pi * i0(kappa))

def build_within_weights():
    dphi = PREFERRED[:, None] - PREFERRED[None, :]
    return (-J_0 + J_1 * np.cos(dphi)) / N

W_WITHIN = build_within_weights()

def decode(response):
    """Cosine similarity decode → angle."""
    n_grid = 500
    theta_grid = np.linspace(-np.pi, np.pi, n_grid, endpoint=False)
    templates = np.array([[tuning_curve(t, phi, KAPPA) for phi in PREFERRED]
                          for t in theta_grid])
    r_norm = response / (np.linalg.norm(response) + 1e-12)
    t_norms = templates / (np.linalg.norm(templates, axis=1, keepdims=True) + 1e-12)
    sims = t_norms @ r_norm
    return theta_grid[np.argmax(sims)]

print(f"Model: N={N}, J_0={J_0}, J_1={J_1}, β={BETA}, h0={H0}")
print(f"Noise σ={NOISE_SIGMA}, steps={N_STEPS}, stim_steps={STIM_STEPS}")
print(f"W_WITHIN shape: {W_WITHIN.shape}")

In [ ]:
# Cell 2: Trial runner with failure mode classification
def run_trial_with_classification(J_cross, input_gain):
    """
    Run one stochastic trial. Returns dict with:
      - error: decoded angular error relative to item 1
      - is_swap: bool (error within 0.3 rad of item 2)
      - max_rA, max_rB: peak firing rate of each network at end
      - failure_mode: 'selection' | 'representation' | 'total_collapse' | None
      - D: dominance variable at end
      - status: 'ok' | 'nan' | 'explosion' | 'collapse'
    """
    theta1, theta2 = 0.0, SEPARATION

    drive_A = input_gain * np.array([tuning_curve(theta1, phi, KAPPA) for phi in PREFERRED])
    drive_B = input_gain * np.array([tuning_curve(theta2, phi, KAPPA) for phi in PREFERRED])

    r_A = np.ones(N) * 0.1
    r_B = np.ones(N) * 0.1

    for step in range(N_STEPS):
        ext_A = drive_A if step < STIM_STEPS else np.zeros(N)
        ext_B = drive_B if step < STIM_STEPS else np.zeros(N)

        cross_A = -J_cross * np.mean(r_B)
        cross_B = -J_cross * np.mean(r_A)

        h_A = W_WITHIN @ r_A + ext_A + cross_A + np.random.randn(N) * NOISE_SIGMA
        r_A = np.maximum(0, r_A + (-r_A + sigmoid(h_A)) * (DT / TAU))

        h_B = W_WITHIN @ r_B + ext_B + cross_B + np.random.randn(N) * NOISE_SIGMA
        r_B = np.maximum(0, r_B + (-r_B + sigmoid(h_B)) * (DT / TAU))

        if np.any(np.isnan(r_A)) or np.any(np.isnan(r_B)):
            return {'status': 'nan'}
        if np.max(r_A) > 1e6 or np.max(r_B) > 1e6:
            return {'status': 'explosion'}

    if np.max(r_A) < 0.01 and np.max(r_B) < 0.01:
        return {'status': 'collapse'}

    # Decode
    combined = r_A + r_B
    decoded = decode(combined)
    error = (decoded - theta1 + np.pi) % (2 * np.pi) - np.pi
    is_swap = abs(error - SEPARATION) < 0.3

    # Failure mode classification (paper §3.5.4 diagnostic)
    max_rA = float(np.max(r_A))
    max_rB = float(np.max(r_B))
    D = float(np.mean(r_A) - np.mean(r_B))

    failure_mode = None
    if is_swap:
        if max_rA > 0.3 and max_rB > 0.3:
            failure_mode = 'selection'       # Both bumps survived, wrong decoded
        elif max_rA > 0.3 or max_rB > 0.3:
            failure_mode = 'representation'  # One bump collapsed
        else:
            failure_mode = 'total_collapse'

    return {
        'status': 'ok',
        'error': float(error),
        'is_swap': bool(is_swap),
        'max_rA': max_rA,
        'max_rB': max_rB,
        'D': D,
        'failure_mode': failure_mode,
    }

# Quick sanity check
np.random.seed(42)
test = run_trial_with_classification(1.0, 6.0)
print(f"Sanity check: {test}")

In [ ]:
# Cell 3: Sweep with progress tracking
J_CROSS_VALUES = [0.1, 0.2, 0.25, 0.3, 0.5, 0.8, 1.0, 1.2, 1.4, 1.6, 2.0, 4.0]
DRIVE_VALUES = [2.0, 4.0, 6.0, 8.0]
N_TRIALS = 500

results = []
total_points = len(J_CROSS_VALUES) * len(DRIVE_VALUES)

t0 = time.time()
for pi, (jx, drv) in enumerate([(j, d) for j in J_CROSS_VALUES for d in DRIVE_VALUES]):
    point = {
        'J_cross': jx, 'drive': drv,
        'n_valid': 0, 'n_swap': 0,
        'n_selection': 0, 'n_representation': 0, 'n_total_collapse': 0,
        'swap_rate': 0.0,
        'errors': [],
    }

    for trial in range(N_TRIALS):
        result = run_trial_with_classification(jx, drv)
        if result['status'] != 'ok':
            continue
        point['n_valid'] += 1
        if result['is_swap']:
            point['n_swap'] += 1
            if result['failure_mode'] == 'selection':
                point['n_selection'] += 1
            elif result['failure_mode'] == 'representation':
                point['n_representation'] += 1
            elif result['failure_mode'] == 'total_collapse':
                point['n_total_collapse'] += 1
        point['errors'].append(result['error'])

    if point['n_valid'] > 0:
        point['swap_rate'] = point['n_swap'] / point['n_valid']
    if point['n_swap'] > 0:
        point['selection_fraction'] = point['n_selection'] / point['n_swap']
        point['representation_fraction'] = point['n_representation'] / point['n_swap']
    else:
        point['selection_fraction'] = 0.0
        point['representation_fraction'] = 0.0

    results.append(point)

    elapsed = time.time() - t0
    eta = elapsed / (pi + 1) * (total_points - pi - 1)
    print(f"[{pi+1}/{total_points}] J_×={jx:.2f}, drive={drv:.1f} → "
          f"swap={point['swap_rate']:.1%} "
          f"(sel={point['selection_fraction']:.0%} rep={point['representation_fraction']:.0%}) "
          f"[ETA {eta:.0f}s]")

print(f"\nDone in {time.time()-t0:.1f}s")

In [ ]:
# Cell 4: Save results as JSON (download from Colab)
# Strip non-serializable lists for compact JSON
results_clean = []
for r in results:
    rc = {k: v for k, v in r.items() if k != 'errors'}
    results_clean.append(rc)

with open('failure_mode_results.json', 'w') as f:
    json.dump(results_clean, f, indent=2)

print("Saved to failure_mode_results.json")
# In Colab: from google.colab import files; files.download('failure_mode_results.json')

In [ ]:
# Cell 5: Generate Figure 10 — Selection vs Representation Failure
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Panel A: Stacked bar by J_cross (averaged over drives)
jx_vals = sorted(set(r['J_cross'] for r in results))
for ax_idx, drive_label in enumerate([(2.0, 'Drive=2'), (6.0, 'Drive=6'), (8.0, 'Drive=8')]):
    drv, label = drive_label
    ax = axes[ax_idx]

    sel_fracs, rep_fracs, swap_rates = [], [], []
    for jx in jx_vals:
        pts = [r for r in results if r['J_cross'] == jx and r['drive'] == drv]
        if pts and pts[0]['n_swap'] > 0:
            sel_fracs.append(pts[0]['selection_fraction'])
            rep_fracs.append(pts[0]['representation_fraction'])
            swap_rates.append(pts[0]['swap_rate'])
        else:
            sel_fracs.append(0)
            rep_fracs.append(0)
            swap_rates.append(0)

    x = np.arange(len(jx_vals))
    width = 0.6

    # Stacked bar: selection (blue) + representation (red)
    bars_sel = [s * sr for s, sr in zip(sel_fracs, swap_rates)]
    bars_rep = [r * sr for r, sr in zip(rep_fracs, swap_rates)]

    ax.bar(x, bars_sel, width, label='Selection failure', color='#2d5a7b', alpha=0.85)
    ax.bar(x, bars_rep, width, bottom=bars_sel, label='Representation failure', color='#c0392b', alpha=0.85)

    ax.set_xticks(x)
    ax.set_xticklabels([f'{v:.1f}' if v < 1 else f'{v:.0f}' for v in jx_vals],
                        rotation=45, fontsize=8)
    ax.set_xlabel('$J_{\\times}$')
    ax.set_ylabel('Swap error rate')
    ax.set_title(label)
    ax.set_ylim(0, 0.55)
    if ax_idx == 0:
        ax.legend(fontsize=8)

    # Mark valley region
    valley_start = jx_vals.index(1.0) if 1.0 in jx_vals else 0
    valley_end = jx_vals.index(1.6) if 1.6 in jx_vals else len(jx_vals)-1
    ax.axvspan(x[valley_start]-0.3, x[valley_end]+0.3,
               alpha=0.08, color='green', label='Valley' if ax_idx==0 else None)

plt.suptitle('Figure 10. Selection vs Representation Failure Across $J_{\\times}$',
             fontsize=13, y=1.02)
plt.tight_layout()
plt.savefig('fig10_failure_modes.png', dpi=200, bbox_inches='tight')
plt.show()
print("Saved fig10_failure_modes.png")

In [ ]:
# Cell 6: Left/right eigenvector check (C5 — quick, can also run locally)
from scipy.optimize import fsolve

def find_fp_and_jacobian(J_cross):
    """Find coexistence FP and compute Jacobian."""
    # Strong-drive initialization
    r_init = np.ones(2*N) * 0.3
    r_init[:N] += 0.3 * np.cos(PREFERRED - np.pi/4)   # bump in A
    r_init[N:] += 0.3 * np.cos(PREFERRED - np.pi/4)   # bump in B

    def F(r):
        rA, rB = r[:N], r[N:]
        hA = W_WITHIN @ rA - J_cross * np.mean(rB)
        hB = W_WITHIN @ rB - J_cross * np.mean(rA)
        return np.concatenate([-rA + sigmoid(hA), -rB + sigmoid(hB)])

    r_star = fsolve(F, r_init, full_output=False)
    rA, rB = r_star[:N], r_star[N:]

    # Jacobian
    hA = W_WITHIN @ rA - J_cross * np.mean(rB)
    hB = W_WITHIN @ rB - J_cross * np.mean(rA)
    sA = BETA * sigmoid(hA) * (1 - sigmoid(hA) / R_MAX)
    sB = BETA * sigmoid(hB) * (1 - sigmoid(hB) / R_MAX)
    SA = np.diag(sA)
    SB = np.diag(sB)
    C = -J_cross / N * np.ones((N, N))

    J = np.block([
        [-np.eye(N) + SA @ W_WITHIN, SA @ C],
        [SB @ C, -np.eye(N) + SB @ W_WITHIN]
    ])
    return r_star, J

r_star, J = find_fp_and_jacobian(0.34)
evals_r, evecs_r = np.linalg.eig(J)        # right eigenvectors
evals_l, evecs_l = np.linalg.eig(J.T)      # left eigenvectors (from J^T)

# Find dominant non-Goldstone
idx_r = np.argsort(-evals_r.real)
for i in idx_r:
    if abs(evals_r[i].real) > 1e-3:  # skip Goldstones
        dom_idx_r = i
        break

# Match left eigenvector by eigenvalue proximity
dom_eval = evals_r[dom_idx_r].real
diffs = np.abs(evals_l.real - dom_eval)
dom_idx_l = np.argmin(diffs)

v_right = evecs_r[:, dom_idx_r].real
v_right /= np.linalg.norm(v_right)
v_left = evecs_l[:, dom_idx_l].real
v_left /= np.linalg.norm(v_left)

overlap = abs(np.dot(v_right, v_left))
print(f"λ_dom = {evals_r[dom_idx_r].real:.6f}")
print(f"|⟨v_right, v_left⟩| = {overlap:.6f}")
print(f"→ {'Negligible difference' if overlap > 0.95 else 'SIGNIFICANT — use left eigenvector!'}")

In [ ]:
# Cell 7: Summary table
print("\n" + "="*80)
print("FAILURE MODE DECOMPOSITION SUMMARY")
print("="*80)
print(f"{'J_×':>6} {'Drive':>6} {'Swap%':>6} {'N_swap':>7} {'Sel%':>6} {'Rep%':>6} {'N_sel':>6} {'N_rep':>6}")
print("-"*55)
for r in results:
    print(f"{r['J_cross']:6.2f} {r['drive']:6.1f} {r['swap_rate']:5.1%} "
          f"{r['n_swap']:7d} {r['selection_fraction']:5.0%} "
          f"{r['representation_fraction']:5.0%} {r['n_selection']:6d} {r['n_representation']:6d}")